# Anak Agung Ngurah Bagus Trihatmaja


# Problem 6.1

> Write a code in Python whose input is a training dataset {(x1, y1), . . . , (xN , yN )} and its output is the weight vector θ in the linear regression model y = θ'φ(x), for a given nonlinear mapping φ(·). Implement two cases: i) using the closed-form solution, ii) using a stochastic gradient descent on mini-batches of size m.

In [1]:
import numpy as np
from numpy.linalg import inv
import scipy.io
import math
import pandas as pd
import time

# For reproducibility
np.random.seed(1000)

In [33]:
# This function generates data based on the formula we input 
# For this time we will generate a square function added with a noise
f = lambda  x: x**2

In [385]:
# Closed function regression
def closed_function_regression(train_X, train_Y):
  X = np.array(train_X)
  y = np.array(train_Y)
  
  
  # TODO: Check again here  
  # ones = np.ones(len(X))
  # X = np.column_stack((ones, X))
  y = np.array(y)
        
  Xt = transpose(X)
  product = dot(Xt, X)
  theInverse = inv(product)
  w = dot(dot(theInverse, Xt), y)
        
  return w       

In [13]:
def shuffle_data(train_X, train_Y, batch_size):
    X = train_X
    y = train_Y
    
    random_indices = np.random.choice(len(X), len(y), replace=False)
    
    X_shuffled = X[random_indices,:]
    y_shuffled = y[random_indices]
    mini_batches = [(X_shuffled[i:i+batch_size,:], y_shuffled[i:i+batch_size]) for
                   i in range(0, len(y), batch_size)]
    
    return mini_batches
    

def mini_gradient_descent(train_X, train_Y, learning_rate, num_iter, batch_size):
    # Prepare the data
    X = np.array(train_X)
    y = np.array(train_Y)
    
    # TODO: Check why do we need to put 1 here
    # ones = np.ones(len(X))
    # X = np.column_stack((ones, X))
    # y = np.array(y)
    
    # Randomize the data and split into samples
    df = shuffle_data(X, y, batch_size)
    
    # Get the dimension
    x_col = X.shape[1]
    
    # Set the initial theta and other initial variables
    theta = np.zeros((x_col, 1))
    
    start_i = 0
    
    # Create the loop
    for i in range(start_i + 1, num_iter + 1):
        # For each randomized-mini-batch partition
        for j in range(0, len(df)):
            X = df[j][0]
            y = df[j][1]
            
            y_hat = np.dot(X, theta)
            
            # Break if overflow occurs
            theta = theta - learning_rate * np.dot(X.T, y_hat - y)
            
            # FIXME: Break if NaN occurs
            # if not (float('-inf') < float(temp[0]) < float('inf')): 
            #    return theta
                
            # theta =  temp
            
        
    return theta

# Problem 6.2

> Consider n-degree polynomials, φ(·) =  1 x x^2 · · · x^n . Download the dataset on the course webpage and work with ‘dataset1’. Run the code on the training data to compute θ for n ∈ {2, 3, 5}. Evaluate the regression error on both training and the test data. Report θ, training error and test error for both implementation (closed-form vs gradient descent). What is the effect of the size of the mini-batch on the speed and testing error of the solution.


In [548]:
df = scipy.io.loadmat("/Users/bagustrihatmaja/Downloads/HW1_Data/dataset1.mat")


X_trn = df['X_trn']
Y_trn = df['Y_trn']
X_test = df['X_tst']
Y_test = df['Y_tst']


closed_function_regression(X_trn, Y_trn)

array([[ 5.40148277]])

In [2]:
def create_polinomial_array(element, n):
    pol = np.empty([1, n+1])
    # Debug
    # print(pol)
    for i in range(0, n+1):
        # Debug
        # print("Element: \n", element)
        pol[0][i] = element ** i
        
    return pol[0]



In [ ]:
# Here we set the degree of polynomial        
degree = 5
# degree + 1 for 1's in the first column
X_trn_new = np.ndarray((len(X_trn), degree + 1))

# print(X_trn_new)

i = 0
X = np.array(X_trn)
for x in X:
    X_trn_new[i] = create_polinomial_array(x, degree)
    i += 1
        

mini_gradient_descent(X_trn, Y_trn, 0.001, 100, 10)

In [575]:
# Analysing the regression error and speed
# Hyper-parameter are the alpha (learning rate), the batch size and the number of iterations
def analyze(learning_rate, n_iteration, batch_size):
    degrees = [2, 3, 5]
    
    # Time for performance
    grad_desc_performance = []
    
    # For train data
    error_closed_form_train = []
    error_grad_desc_train = []
    
    # For test data
    error_closed_form_test = []
    error_grad_desc_test = []
    
    X = np.array(X_trn)
    X_tst = np.array(X_test)
    
    for degree in degrees:
        X_trn_new = np.ndarray((len(X_trn), degree + 1))
        X_tst_new = np.ndarray((len(X_tst), degree + 1))
        
        i = 0
        for x in X:
            X_trn_new[i] = create_polinomial_array(x, degree)
            i += 1
        
        i = 0
        for x in X_tst:
            X_tst_new[i] = create_polinomial_array(x, degree)
            i += 1
        
        tetha_closed_form = closed_function_regression(X_trn_new, Y_trn)
        t = time.process_time()
        tetha_grad_desc = mini_gradient_descent(X_trn_new, Y_trn, learning_rate, n_iteration, batch_size)
        elapsed_time = time.process_time() - t
        print("Theta for degree {} done in: {}\n".format(degree, elapsed_time))
        
        
        y_hat_closed_form_train = np.dot(X_trn_new, tetha_closed_form)
        y_hat_grad_desc_train = np.dot(X_trn_new, tetha_grad_desc)
        
        y_hat_closed_form_test = np.dot(X_tst_new, tetha_closed_form)
        y_hat_grad_desc_test = np.dot(X_tst_new, tetha_grad_desc)
        
        error_closed_form_train.append(np.sqrt(((Y_trn - y_hat_closed_form_train) ** 2).mean()))
        error_grad_desc_train.append(np.sqrt(((Y_trn - y_hat_grad_desc_train) ** 2).mean()))
        
        error_closed_form_test.append(np.sqrt(((Y_test - y_hat_closed_form_test) ** 2).mean()))
        error_grad_desc_test.append(np.sqrt(((Y_test - y_hat_grad_desc_test) ** 2).mean()))
        
    d = {
        'degree': [2, 3, 5],
        'close_form_error_train': error_closed_form_train,
        'grad_desc_error_train': error_grad_desc_train,
        'close_form_error_test': error_closed_form_test,
        'grad_desc_error_test': error_grad_desc_test
    }
    errors = pd.DataFrame(data = d)
    return errors




In [576]:
# Batch size 10
result1 = analyze(0.00001, 10, 10)
result1

Theta for degree 2 done in: 0.007232999999999379

Theta for degree 3 done in: 0.0033169999999955735

Theta for degree 5 done in: 0.00168699999998978



,close_form_error_test,close_form_error_train,degree,grad_desc_error_test,grad_desc_error_train
0,66.434831,4.974125,2,7.538616e+01,1.652118e+01
1,7.336971,1.991950,3,1.865774e+01,1.167766e+01
2,6.442782,1.986676,5,5.080623e+90,2.000555e+89


In [577]:
# Batch size 20
result1 = analyze(0.00001, 6, 20)
result1

Theta for degree 2 done in: 0.0021930000000054406

Theta for degree 3 done in: 0.0027790000000038617

Theta for degree 5 done in: 0.0026310000000080436



,close_form_error_test,close_form_error_train,degree,grad_desc_error_test,grad_desc_error_train
0,66.434831,4.974125,2,7.064596e+01,1.862262e+01
1,7.336971,1.991950,3,2.631678e+01,1.405648e+01
2,6.442782,1.986676,5,6.622336e+46,2.415631e+45


With the hyperparameter of `number of iteration` is 10 and `learning rate` is 0.00001 and we split the data by 10, so we get 12 partition. We get smaller RMSE compared to the closed form for our training data.

The error we get varies depends on our hyperparameters we mention above. In case of polinomial of 5, we get better result if we increase the batch size and adjust the number of iteration accordingly.

With batch size of 10, we get the time decreasing from degree 2, 3 to 5 but for batch size of 20, the time is similar, except for degree 2. Therefore, we can conclude there is no significant influence of the batch time to the performance in this case.

We have known bug that if we set the learning rate or the number of iteration too high, we will overflow exception (known bugs are marked as `# FIXME` in this report).

# Problem 6.3

> Download the dataset on the course webpage and work with ‘dataset2’. Write a code in Python that applies Ridge regression to the dataset to compute θ for a given λ. Implement two cases:
 2
using a closed-form solution and using a stochastic gradient descent method with mini-batches of size m. Use K-fold cross validation on the training dataset to obtain the best regularization λ and apply the optimal θ to compute the regression error on test samples. Report the optimal λ, θ, test and training set errors for K ∈ {2,10,N}, where N is the number of samples. In all cases try n ∈ {2, 3, 5}. How does the test error change as a function of λ and n?

In [24]:
class RidgeRegression(object):
    def __init__(self, lmbda=0.1):
        self.lmbda = lmbda
    
    def fit(self, X, y):
        C = X.T.dot(X) + self.lmbda * np.eye(X.shape[1])
        self.w = np.linalg.inv(C).dot(X.T.dot(y))
    
    def set_params(self, lmbda=0.1):
        self.lmbda = lmbda
        return self
        
    def get_weight(self):
        return self.w
    
    # def get_error(self):
    #     y_hat = np.dot(self.X, self.w)
    #     return np.sqrt(((self.Y - y_hat) ** 2).mean())

In [16]:
class Minimisation(object):
    def __init__(self, X, y, model):
        self.model = model
        # Prepare the data
        self.X = np.array(X)
        self.y = np.array(y)
        
    def closed_form(self):
        self.model.fit(self.X, self.y)
        return self.model.get_weight()
    
    def mini_batch_stochastic(self, batch_size):
        y = np.array(y)
    
        # Randomize the data and split into samples
        df = shuffle_data(X, y, batch_size)
    
        # Get the dimension
        x_dim = X.shape[1]
    
        # Set the initial tetha and other initial variables
        theta = np.zeros((x_dim, 1))
    
        start_i = 0
    
        # Create the loop
        for i in range(start_i + 1, num_iter + 1):
            # For each randomized-mini-batch partition
            for j in range(0, len(df)):
                X = df[j][0]
                y = df[j][1]
                
                self.model.fit(self.X, self.y)
                theta = theta - learning_rate * self.model.get_weight 
        
        return theta
        
    def shuffle_data(self, batch_size):
        X = self.X
        y = self.y
    
        random_indices = np.random.choice(len(X), len(y), replace=False)
    
        X_shuffled = X[random_indices,:]
        y_shuffled = y[random_indices]
        mini_batches = [(X_shuffled[i:i+batch_size,:], y_shuffled[i:i+batch_size]) for
                   i in range(0, len(y), batch_size)]
        return mini_batches


In [17]:
df2 = scipy.io.loadmat("/Users/bagustrihatmaja/Downloads/HW1_Data/dataset2.mat")


X_trn2 = df2['X_trn']
Y_trn2 = df2['Y_trn']



X_test2 = df2['X_tst']
Y_test2 = df2['Y_tst']


ridge = RidgeRegression()
m = Minimisation(X_trn2, Y_trn2, ridge)

In [31]:
def k_fold(X, y, lmbda, fold):
    # Split the data into fold part
    # Prepare the data
    X = np.array(X)
    y = np.array(y)
    ridge = RidgeRegression()
    
    # Randomize the data and split into samples
    batch_size = int(len(X) / fold)
    df = shuffle_data(X, y, batch_size)
    
    test_result = []
    
    for i in range(0, len(df)):
        # Temporary data for k-fold
        temp = df.copy()
        
        # To store partitions for training
        train = []
        
        # Save and pop an element for test
        test = temp.pop(i)
        
        # Concat the remaining array
        X_train = np.empty(shape=[X.shape[0], X.shape[1]])
        Y_train = np.empty(shape=[y.shape[0], y.shape[1]])
        
        for j in range(0, len(df)):
            _X = df[j][0]
            _y = df[j][1]
            X_train = np.concatenate((X_train, _X))
            Y_train = np.concatenate((Y_train, _y))
        
        
        ridge.set_params(lmbda)
        # test[0] is X in test data
        # test[1] is Y in test data
        
                            
        ridge.fit(X_train, Y_train)
        #print(X_train, Y_train)
        w = ridge.get_weight()
        
        if ~(math.isnan(w[0])):
            y_hat = np.dot(test[0], w)
            test_result.append(np.sqrt(((test[1] - y_hat) ** 2).mean()))
    
    return np.nanmean(test_result)

    
 


In [35]:
def k_fold_analysis(degree, fold):
    X = np.array(X_trn2)
    k_fold_result = []
    
    X_trn_new = np.ndarray((len(X_trn2), degree + 1))
    i = 0
    for x in X:
        X_trn_new[i] = create_polinomial_array(x, degree)
        i += 1
        
    
    for j in range(-10, 2):
        k_fold_result.append(k_fold(X_trn_new, Y_trn2, 10 ** j, fold))
    
    return k_fold_result


In [40]:
degrees = [2, 3, 5]
folds = [2, 10, 20]

for degree in degrees:
    for fold in folds:
        temp = k_fold_analysis(degree, fold)
        print("Polynomial degree: {}, K-Fold: {}\n".format(degree, fold))
        print("Minimum error at index: {} value {} with lambda: 10^{}\n".format(
            np.argmin(temp), np.min(temp), (np.argmin(temp) - 10)))
    

Polynomial degree: 2, K-Fold: 2

Minimum error at index: 2 value 63.90616407539763 with lambda: 10^-8

Polynomial degree: 2, K-Fold: 10

Minimum error at index: 1 value 40.74383837886053 with lambda: 10^-9

Polynomial degree: 2, K-Fold: 20

Minimum error at index: 9 value 58.46242254776773 with lambda: 10^-1

Polynomial degree: 3, K-Fold: 2

Minimum error at index: 7 value 15.758953693836744 with lambda: 10^-3

Polynomial degree: 3, K-Fold: 10

Minimum error at index: 6 value 30.238897004998698 with lambda: 10^-4

Polynomial degree: 3, K-Fold: 20

Minimum error at index: 3 value 26.10703357540074 with lambda: 10^-7



Polynomial degree: 5, K-Fold: 2

Minimum error at index: 0 value 15.561790112626733 with lambda: 10^-10

Polynomial degree: 5, K-Fold: 10

Minimum error at index: 2 value 29.488860256919907 with lambda: 10^-8

Polynomial degree: 5, K-Fold: 20

Minimum error at index: 6 value 15.644992536061014 with lambda: 10^-4



From the above result, we see that the best lambda is depending upon the number of parameters in X--polynomial with degree 5 has different lambda that result the smallest error compared to other polynomial degree.